In [ ]:
# ==============================================================================
#  This code was written by himmeow the coder.
#  Contact: himmeow.thecoder@gmail.com
#  Discord server: https://discord.gg/deua7trgXc
#
#  Feel free to use and modify this code as you see fit.
#  If you find it helpful, I'd appreciate a coffee!
#  - Momo: 0374525177
#  - Vietcombank (VCB): 1014622635
# ==============================================================================

## Cài đặt môi trường

In [2]:
!pip install transformers torch accelerate
!pip install langchain_openai langchain_community
!pip install faiss-cpu gdown

/media/mtl/DATA 6TB5/PROJECT AI/RAG-NLP/.venv/bin/pip: 2: exec: /media/mtl/DATA 6TB/PROJECT AI/RAG-NLP/.venv/bin/python: not found
/media/mtl/DATA 6TB5/PROJECT AI/RAG-NLP/.venv/bin/pip: 2: exec: /media/mtl/DATA 6TB/PROJECT AI/RAG-NLP/.venv/bin/python: not found
/media/mtl/DATA 6TB5/PROJECT AI/RAG-NLP/.venv/bin/pip: 2: exec: /media/mtl/DATA 6TB/PROJECT AI/RAG-NLP/.venv/bin/python: not found


In [2]:
!pip install gdown -U --no-cache-dir

/media/mtl/DATA 6TB5/PROJECT AI/RAG-NLP/.venv/bin/pip: 2: exec: /media/mtl/DATA 6TB/PROJECT AI/RAG-NLP/.venv/bin/python: not found


In [5]:
# Thư viện để tải dữ liệu từ Google Drive
import gdown

# URL thư mục chứa dữ liệu trên Google Drive
url = "https://drive.google.com/drive/folders/1K13Lo8sX5iRXJ-rNsiwB-cuHTTz3NdP5?usp=drive_link"
# Tải dữ liệu từ Google Drive
gdown.download_folder(url)

ModuleNotFoundError: No module named 'gdown'

In [ ]:
import os

# Thiết lập API key cho OpenAI
os.environ['OPENAI_API_KEY'] = "sk-..."

# Thư viện để tạo vector embedding từ OpenAI
from langchain_openai import OpenAIEmbeddings
# Thư viện để chia văn bản thành các đoạn nhỏ
from langchain.text_splitter import TextSplitter
# Thư viện để tạo và quản lý cơ sở dữ liệu vector
from langchain_community.vectorstores import FAISS

# Định nghĩa class để chia văn bản theo dòng
class LineTextSplitter(TextSplitter):
    def split_text(self, text):
        return text.split('\n')

# Khởi tạo đối tượng TextSplitter
text_splitter = LineTextSplitter()

## Xử lý dữ liệu

### Xử lý dữ liệu thông tin nhân viên

In [ ]:
# Đọc dữ liệu từ file
with open('/content/data/employee_info.txt', 'r', encoding='utf-8') as f:
    text = f.read()
# Chia văn bản thành các đoạn theo dòng
documents = text_splitter.split_text(text)
# Tạo cơ sở dữ liệu vector từ các đoạn văn bản
db = FAISS.from_texts(documents, OpenAIEmbeddings(model="text-embedding-3-large"))
# Lưu cơ sở dữ liệu vào file
db.save_local("/content/index/employee_index")

### Xử lý dữ liệu sản phẩm

In [ ]:
with open('/content/data/products.txt', 'r', encoding='utf-8') as f:
    text = f.read()
documents = text_splitter.split_text(text)
db = FAISS.from_texts(documents, OpenAIEmbeddings(model="text-embedding-3-large"))
db.save_local("/content/index/products_index")

### Xử lý dữ liệu đánh giá

In [ ]:
with open('/content/data/reviews.txt', 'r', encoding='utf-8') as f:
    text = f.read()
documents = text_splitter.split_text(text)
db = FAISS.from_texts(documents, OpenAIEmbeddings(model="text-embedding-3-large"))
db.save_local("/content/index/reviews_index")

## Định nghĩa các hàm truy vấn thông tin

In [ ]:
def about_employee(info):
    """
    Cung cấp thông tin về các thành viên ban quản lý và nhân viên của công ty

    Args:
        info (str): Thông tin cần tìm kiếm
    Returns:
        str: Chuỗi JSON chứa thông tin tìm thấy
    """
    # Load cơ sở dữ liệu thông tin nhân viên
    db = FAISS.load_local("index/employee_index", OpenAIEmbeddings(model="text-embedding-3-large"), allow_dangerous_deserialization = True)
    # Tìm kiếm thông tin tương đồng
    results = db.similarity_search(info, k=1)
    # Chuyển đổi kết quả thành JSON
    docs = [{"content": doc.page_content} for doc in results]
    docs_string = json.dumps(docs, ensure_ascii=False)

    return docs_string

def about_products(info):
    """
    Cung cấp thông tin về các sản phẩm đang có ở công ty

    Args:
        info (str): Thông tin cần tìm kiếm
    Returns:
        str: Chuỗi JSON chứa thông tin tìm thấy
    """
    db = FAISS.load_local("index/products_index", OpenAIEmbeddings(model="text-embedding-3-large"), allow_dangerous_deserialization = True)
    results = db.similarity_search(info, k=3)
    docs = [{"content": doc.page_content} for doc in results]
    docs_string = json.dumps(docs, ensure_ascii=False)

    return docs_string

def reviews_search(info):
    """
    Cung cấp đánh giá của người dùng về những sản phẩm của công ty

    Args:
        info (str): Thông tin cần tìm kiếm
    Returns:
        str: Chuỗi JSON chứa thông tin tìm thấy
    """
    db = FAISS.load_local("index/reviews_index", OpenAIEmbeddings(model="text-embedding-3-large"), allow_dangerous_deserialization = True)
    results = db.similarity_search(info, k=5)
    docs = [{"content": doc.page_content} for doc in results]
    docs_string = json.dumps(docs, ensure_ascii=False)

    return docs_string

## Định nghĩa metadata cho các hàm

In [ ]:
import json

functions_metadata = [
    {
        "type": "function",
        "function": {
            "name": "about_employee",
            "description": "Cung cấp những tài liệu liên quan đến người quản lý/nhân viên của công ty mà bạn cần biết.",
            "parameters": {
                "type": "object",
                "properties": {
                    "info": {
                        "type": "string",
                        "description": "Thông tin mà bạn cần tìm kiếm, e.g. Email của himmeow the coder.",
                    },
                },
                "required": ["info"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "about_products",
            "description": "Cung cấp những tài liệu liên quan đến sản phẩm của công ty mà bạn cần biết.",
            "parameters": {
                "type": "object",
                "properties": {
                    "info": {
                        "type": "string",
                        "description": "Thông tin mà bạn cần tìm kiếm, e.g. Xuất xứ của áo phông nam Luôn Vui Tươi.",
                    },
                },
                "required": ["info"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "reviews_search",
            "description": "Cung cấp những lời nhận xét về các sản phẩm của công ty.",
            "parameters": {
                "type": "object",
                "properties": {
                    "info": {
                        "type": "string",
                        "description": "Những nhận xét mà bạn cần tìm kiếm, e.g. Nhận xét của người dùng về sản phẩm áo phông nam.",
                    },
                },
                "required": ["info"],
            },
        },
    },
]

## Khởi tạo bộ nhớ cho chatbot

In [ ]:
messages = [
    {"role": "system", "content": f"""Bạn là một trợ lý hữu ích của một cửa hàng bán đồ dùng gia đình, có quyền truy cập vào các chức năng sau, sử dụng chúng nếu cần -\n{str(functions_metadata)} Để sử dụng các chức năng này, hãy phản hồi với:\n<functioncall> {{\\"name\\": \\"function_name\\", \\"arguments\\": {{\\"arg_1\\": \\"value_1\\", \\"arg_1\\": \\"value_1\\", ...}} }} </functioncall>\n\nTrường hợp đặc biệt bạn phải xử lý:\n - Nếu không có chức năng nào khớp với yêu cầu của người dùng, bạn sẽ phản hồi một cách lịch sự rằng bạn không thể giúp được.""" },
]

## Tải model và tokenizer từ Hugging Face

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained('hiieu/Vistral-7B-Chat-function-calling')
model = AutoModelForCausalLM.from_pretrained(
    'hiieu/Vistral-7B-Chat-function-calling',
    torch_dtype=torch.bfloat16, # Thay đổi thành torch.float16 nếu sử dụng V100
    device_map="auto",
    use_cache=True,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.57k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/597k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.15M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/83.0 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/133 [00:00<?, ?B/s]

## Định nghĩa các hàm hỗ trợ

In [ ]:
def remove_extra_quotes(function_call_string):
  """
  Loại bỏ dấu nháy đơn thừa trong chuỗi function call.

  Args:
    function_call_string: Chuỗi function call có thể chứa dấu nháy đơn thừa.

  Returns:
    Chuỗi function call đã được xử lý, hoặc None nếu không tìm thấy JSON hợp lệ.
  """
  start = function_call_string.find('{')
  end = function_call_string.rfind('}')
  if start != -1 and end != -1:
    json_string = function_call_string[start:end+1]
    try:
      # Thử phân tích JSON để kiểm tra xem có hợp lệ hay không
      json.loads(json_string)
      return function_call_string
    except json.JSONDecodeError:
      # Nếu không hợp lệ, thử loại bỏ dấu nháy đơn thừa
      json_string = json_string.replace("'{", "{").replace("'}", "}")
      # Kiểm tra lại xem JSON đã hợp lệ chưa
      try:
        json.loads(json_string)
        return function_call_string[:start] + json_string + function_call_string[end+1:]
      except json.JSONDecodeError:
        return None
  else:
    return None

def remove_simulated_conversation(text):
  """
  Loại bỏ phần mô phỏng cuộc trò chuyện từ chuỗi đầu vào.

  Args:
    text: Chuỗi đầu vào có thể chứa phần mô phỏng cuộc trò chuyện.

  Returns:
    Chuỗi đã được xử lý, loại bỏ phần mô phỏng cuộc trò chuyện.
  """
  index = text.find(".user")
  if index != -1:
    return text[:index + 1]  # Giữ lại dấu "."
  else:
    return text

## Hàm chính để sử dụng mô hình dưới dạng chatbot

In [ ]:
def ask_AI(query):
  # Thêm câu hỏi của người dùng vào bộ nhớ
  messages.append({"role": "user", "content": query})
  # Mã hóa văn bản thành input ids cho model
  input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

  # Sinh text từ model
  outputs = model.generate(
    input_ids,
    max_new_tokens=768,
    do_sample=True,
    top_p=0.95,
    top_k=40,
    temperature=0.1,
    repetition_penalty=1.05,
  )

  # Giải mã output từ model thành text
  response = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)

  # Tìm kiếm function call trong phản hồi
  match = re.search(r"<functioncall>.*</functioncall>", response)

  # Xử lý function call nếu tìm thấy
  if match:
    function_call_string = match.group(0)
    # Thêm function call vào bộ nhớ
    messages.append({"role": "assistant", "content": function_call_string})
    json_match = re.search(r"{.*}", function_call_string)
    if json_match:
      json_string = json_match.group(0)
      json_string = remove_extra_quotes(json_string)
      try:
        function_call = json.loads(json_string)
        function_name = function_call["name"]
        function_args = function_call["arguments"]

        # Gọi hàm tương ứng với function name
        if function_name == "about_employee":
          info = function_args["info"]
          fnc_result = about_employee(info)
          messages.append({"role": "user", "content": f"""<function_response> {fnc_result} </function_response>"""})
        elif function_name == "about_products":
          info = function_args["info"]
          fnc_result = about_products(info)
          messages.append({"role": "user", "content": f"""<function_response> {fnc_result} </function_response>"""})
        elif function_name == "reviews_search":
          info = function_args["info"]
          fnc_result = reviews_search(info)
          messages.append({"role": "user", "content": f"""<function_response> {fnc_result} </function_response>"""})
        else:
          print("Unknown function called.")

        # Gọi lại model với thông tin bổ sung từ function call
        input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
).to(model.device)

        outputs = model.generate(
        input_ids,
        max_new_tokens=768,
        do_sample=True,
        top_p=0.95,
        top_k=40,
        temperature=0.1,
        repetition_penalty=1.05,
        )

        response = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)
        return response

      except json.JSONDecodeError:
        print("Invalid function call format.")

  # Nếu không tìm thấy function call, thêm phản hồi vào bộ nhớ
  else:
    messages.append({"role": "assistant", "content": response})
    return response

## Sử dụng chatbot

In [ ]:
res = ask_AI("xin chào")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
res

' Xin chào! Tôi có thể hỗ trợ gì cho bạn hôm nay? '

In [ ]:
res = ask_AI("Cửa hàng của bạn có sản phẩm nào của thương hiệu SoundWave không?")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
res

' Có, cửa hàng của chúng tôi có sản phẩm của thương hiệu SoundWave. Một trong số đó là tai nghe Bluetooth True Wireless. Nó cung cấp kết nối ổn định và âm thanh chất lượng cao. '